# Persisting series of SIRDS model

In [1]:
import numpy as np
import pandas as pd
import datetime

from sirds_model_visualization import get_sirds, get_sirds_extras

## Reading data

In [2]:
df = pd.read_csv('data/output/df_ts_epidemic_episodes.csv', index_col=0)
df.DATA = pd.to_datetime(df.DATA)

In [3]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df.CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL / df.POPULACAO_2022 * 100000

In [4]:
df_results = pd.read_csv('data/output/estimation_results.csv')
df_results['date_first_case'] = df_results['date_first_case'].str[:11]
df_results.date_first_case = pd.to_datetime(df_results['date_first_case'], errors='coerce')
df_results['date_first_case'] = df_results['date_first_case'].fillna(df_results['date_first_case'].min())

In [5]:
df_results_outbreak_0 = pd.read_csv('data/output/estimation_results_outbreak_0.csv')
df_results_outbreak_0['date_first_case'] = df_results_outbreak_0['date_first_case'].str[:11]
df_results_outbreak_0.date_first_case = pd.to_datetime(df_results_outbreak_0['date_first_case'], errors='coerce')
df_results_outbreak_0['date_first_case'] = df_results_outbreak_0['date_first_case'].fillna(df_results_outbreak_0['date_first_case'].min())

In [6]:
df_results_outbreak_0_days_to_fit_21 = df_results[(df_results.outbreak == 0) & (df_results.alpha == 0)].copy()
df_results_outbreak_0_days_to_fit_21['days_to_fit_within_outbreak'] = 21
df_results_outbreak_0 = pd.concat([df_results_outbreak_0, df_results_outbreak_0_days_to_fit_21])

In [7]:
df_results_outbreak_0 = df_results_outbreak_0.sort_values(['days_to_fit_within_outbreak', 'estimation'])
df_results_outbreak_0.days_to_fit_within_outbreak.unique()

array([21., 28., 35., 42., 49., 56., 63., 70.])

## Executing simulations

In [8]:
def execute_simulations(df_results):
    df_sirds = pd.DataFrame()
    
    for index, result in df_results.iterrows():
        result = result.dropna()
        
        y = get_sirds(result)
        S, I, R, D, I_accumulated = y
        S = S[:]
        I = I[:]
        R = R[:]
        D = D[:]
        I_accumulated = I_accumulated[:]
        N = S[0] + I[0] + R[0] + D[0]
       
        D_new_deaths, reproduction_number_sird, I_new_cases, epidemic_periods_with_fast_transition_fuzzy_variable, epidemic_periods_with_slow_transition_fuzzy_variable, days_between_infections_values, case_fatality_probability_values, loss_immunity_in_days_values, estimated_days_between_infections, estimated_case_fatality_probability, estimated_loss_immunity_in_days = get_sirds_extras(result, S, D, I_accumulated)
        
        date_first_case = result.date_first_case 
        period_in_days = int(result.period_in_days)
        max_date_to_fit = pd.to_datetime(result.max_date_to_fit)
        train_period = (max_date_to_fit - date_first_case).days + 1
        
        date_list = [result.date_first_case + datetime.timedelta(days=i) for i in range(period_in_days)]
        D_new_deaths = np.append([None], D_new_deaths)
        I_new_cases = np.append([None], I_new_cases)
        
        df_temp = pd.DataFrame({
                    'date': date_list,
                    'S': S,
                    'I': I,
                    'R': R,
                    'D': D,
                    'I_accumulated': I_accumulated,
                    'D_new_deaths': D_new_deaths,
                    'reproduction_number_sird': reproduction_number_sird,
                    'I_new_cases': I_new_cases,
                    'days_between_infections': estimated_days_between_infections,
                    'case_fatality_probability': estimated_case_fatality_probability,
                    'loss_immunity_in_days': estimated_loss_immunity_in_days
                })
        
        df_temp['train_period'] = None
        df_temp['train_period'] = df_temp['train_period'].astype(bool)
        df_temp.loc[:train_period,'train_period'] = True
        df_temp.loc[train_period:,'train_period'] = False
        
        df_temp['outbreak'] = result.outbreak
        df_temp['alpha'] = result.alpha
        df_temp['estimation'] = result.estimation
        try:
            df_temp['days_to_fit_within_outbreak'] = result['days_to_fit_within_outbreak']
        except KeyError:
            pass
                    
        df_sirds = pd.concat([df_sirds, df_temp], ignore_index=True)
    
    return df_sirds

In [9]:
df_sirds = execute_simulations(df_results)

In [10]:
df_sirds_outbreak_0 = execute_simulations(df_results_outbreak_0)

In [11]:
df_sirds_outbreak_0

,date,S,I,R,D,I_accumulated,D_new_deaths,reproduction_number_sird,I_new_cases,days_between_infections,case_fatality_probability,loss_immunity_in_days,train_period,outbreak,alpha,estimation,days_to_fit_within_outbreak
0,2020-02-16,99999.988027,0.011973,0.000000,0.000000,0.011973,None,2.515196,None,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
1,2020-02-17,99999.983885,0.014470,0.001637,0.000008,0.016119,0.000008,2.515196,0.004146,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
2,2020-02-18,99999.978885,0.017489,0.003608,0.000018,0.021129,0.00001,2.515196,0.00501,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
3,2020-02-19,99999.972850,0.021136,0.005984,0.000031,0.027183,0.000012,2.515195,0.006054,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
4,2020-02-20,99999.965561,0.025544,0.008850,0.000045,0.034501,0.000015,2.515195,0.007318,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24075,2020-08-04,94988.908760,442.668946,4506.565907,61.856387,5885.482349,0.629194,0.993632,55.002952,7.650937,0.011365,276.577406,False,0.0,0.0,19.0,70.0
24076,2020-08-05,94950.332357,442.285352,4544.897254,62.485037,5940.414437,0.628651,0.993226,54.932088,7.650937,0.011365,276.577406,False,0.0,0.0,19.0,70.0
24077,2020-08-06,94911.969753,441.878785,4583.038352,63.113110,5995.272674,0.628072,0.992823,54.858237,7.650937,0.011365,276.577406,False,0.0,0.0,19.0,70.0
24078,2020-08-07,94873.823043,441.449799,4620.986587,63.740571,6050.054675,0.627461,0.992422,54.782001,7.650937,0.011365,276.577406,False,0.0,0.0,19.0,70.0


## Predicting the end of forecasting

In [12]:
spend_hours = df_results[df_results.alpha==0].duration_seconds.sum()/60/60
print('Spend hours: ', spend_hours)
current_simulations = len(df_results[df_results.alpha==0])
print('current_simulations: ', current_simulations)
hours_per_simulation = spend_hours/current_simulations
print('hours_per_simulation: ', hours_per_simulation)
total_simulations = 1 * 9 * 20
print('total_simulations: ', total_simulations)
estimated_hours_conclusion = (total_simulations-current_simulations)*hours_per_simulation
print('estimated_hours_conclusion: ', estimated_hours_conclusion)
prediction = datetime.datetime.now() + datetime.timedelta(hours=estimated_hours_conclusion)
print('Finish at: ',prediction)

Spend hours:  247.51416726277776
current_simulations:  180
hours_per_simulation:  1.375078707015432
total_simulations:  180
estimated_hours_conclusion:  0.0
Finish at:  2023-12-05 21:51:34.944010


## Saving data

In [13]:
df_sirds.to_csv('data/output/df_sirds.csv', index=False)

In [14]:
df_sirds.head()

,date,S,I,R,D,I_accumulated,D_new_deaths,reproduction_number_sird,I_new_cases,days_between_infections,case_fatality_probability,loss_immunity_in_days,train_period,outbreak,alpha,estimation
0,2020-02-16,99999.988027,0.011973,0.000000,0.000000,0.011973,None,2.515196,None,3.180667,0.005061,250.947207,True,0.0,0.0,0.0
1,2020-02-17,99999.983885,0.014470,0.001637,0.000008,0.016119,0.000008,2.515196,0.004146,3.180667,0.005061,250.947207,True,0.0,0.0,0.0
2,2020-02-18,99999.978885,0.017489,0.003608,0.000018,0.021129,0.00001,2.515196,0.00501,3.180667,0.005061,250.947207,True,0.0,0.0,0.0
3,2020-02-19,99999.972850,0.021136,0.005984,0.000031,0.027183,0.000012,2.515195,0.006054,3.180667,0.005061,250.947207,True,0.0,0.0,0.0
4,2020-02-20,99999.965561,0.025544,0.008850,0.000045,0.034501,0.000015,2.515195,0.007318,3.180667,0.005061,250.947207,True,0.0,0.0,0.0


In [15]:
df_sirds_outbreak_0.to_csv('data/output/df_sirds_outbreak_0.csv', index=False)

In [16]:
df_sirds_outbreak_0.head()

,date,S,I,R,D,I_accumulated,D_new_deaths,reproduction_number_sird,I_new_cases,days_between_infections,case_fatality_probability,loss_immunity_in_days,train_period,outbreak,alpha,estimation,days_to_fit_within_outbreak
0,2020-02-16,99999.988027,0.011973,0.000000,0.000000,0.011973,None,2.515196,None,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
1,2020-02-17,99999.983885,0.014470,0.001637,0.000008,0.016119,0.000008,2.515196,0.004146,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
2,2020-02-18,99999.978885,0.017489,0.003608,0.000018,0.021129,0.00001,2.515196,0.00501,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
3,2020-02-19,99999.972850,0.021136,0.005984,0.000031,0.027183,0.000012,2.515195,0.006054,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
4,2020-02-20,99999.965561,0.025544,0.008850,0.000045,0.034501,0.000015,2.515195,0.007318,3.180667,0.005061,250.947207,True,0.0,0.0,0.0,21.0
